In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/climamm.zip -d /content

Streaming output truncated to the last 5000 lines.
  inflating: /content/climamm/4pxFcm7Cu2.jpg  
  inflating: /content/__MACOSX/climamm/._4pxFcm7Cu2.jpg  
  inflating: /content/climamm/GFttAcjXzG.jpg  
  inflating: /content/__MACOSX/climamm/._GFttAcjXzG.jpg  
  inflating: /content/climamm/PWFvbMDFwE.jpg  
  inflating: /content/__MACOSX/climamm/._PWFvbMDFwE.jpg  
  inflating: /content/climamm/yAjsIklgpf.jpg  
  inflating: /content/climamm/nXxaw8BpBM.jpg  
  inflating: /content/__MACOSX/climamm/._nXxaw8BpBM.jpg  
  inflating: /content/climamm/9rI9tuqRQw.jpg  
  inflating: /content/climamm/MST5y1rmL9.jpg  
  inflating: /content/__MACOSX/climamm/._MST5y1rmL9.jpg  
  inflating: /content/climamm/0IbsoABlML.jpg  
  inflating: /content/__MACOSX/climamm/._0IbsoABlML.jpg  
  inflating: /content/climamm/X59nVm85nI.jpg  
  inflating: /content/__MACOSX/climamm/._X59nVm85nI.jpg  
  inflating: /content/climamm/9Tsjt77MQZ.jpg  
  inflating: /content/__MACOSX/climamm/._9Tsjt77MQZ.jpg  
  inflating: /c

In [4]:
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [5]:
import glob
from itertools import chain
import os
import random
import zipfile

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, utils
from tqdm.notebook import tqdm

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

import seaborn as sns
import matplotlib.pyplot as plt

import timm


In [6]:
# Training settings
batch_size = 16
epochs = 10
lr = 1e-5
gamma = 0.7
weight_decay = 1e-5

seed = 510

# task= 'hate'
# task= 'extremism'
# task= 'stance'
task = 'humor'
# task= 'target'

# for hate and extremism
# NUM_CLASSES= 2

#for stance and humor
NUM_CLASSES= 3

#for target
# NUM_CLASSES= 4

model_used = 'vit'

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [8]:
class ImageDataset(Dataset):
    def __init__(self, data, img_dir, split, transform=None):
        # self.data = data.loc[data['split'] == split]
        self.data = data[data['split'] == split].reset_index(drop=True)

        if task == 'target':
            self.data = self.data[self.data['hate'] == 1].reset_index(drop=True)

        if task == 'extremism':
            self.data = self.data[self.data['stance'].isin([1, 2])].reset_index(drop=True)

        # self.data = data
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.data.loc[idx, 'name'])
        image = Image.open(img_name).convert('RGB')
        label = self.data.loc[idx, task]

        if self.transform:
          image = self.transform(image)

        return (image, label)

In [15]:
train_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(0.3),
        transforms.RandomVerticalFlip(0.3),
        transforms.RandomRotation(120),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)

val_transforms = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)


test_transforms = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)

In [17]:
csv_path = 'drive/MyDrive/climamm.csv'
img_dir = '/content/climamm'

data = pd.read_csv(csv_path)

train_dataset = ImageDataset(data=data, img_dir=img_dir, split='train', transform=train_transforms)
val_dataset = ImageDataset(data=data, img_dir=img_dir, split='val', transform=val_transforms)
test_dataset = ImageDataset(data=data, img_dir=img_dir, split='test', transform=test_transforms)



train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# for image, label in train_dataset:
#     pass

In [18]:
print(len(train_data_loader.dataset))
print(len(val_data_loader.dataset))
print(len(test_data_loader.dataset))

3397
206
398


In [19]:
device = 'cuda'


model = timm.create_model('vit_large_patch14_clip_224.openai', pretrained=True, num_classes=NUM_CLASSES).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [20]:
# timm.list_models(pretrained = True)[200:]

In [21]:


# Train Model

# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay = weight_decay)
# optimizer = optim.Adam(model.parameters(), lr=lr)

# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

model.train()
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_data_loader):
        data = data.to(device)
        label = label.type(torch.LongTensor)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_data_loader)
        epoch_loss += loss / len(train_data_loader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in val_data_loader:
            data = data.to(device)
            label = label.type(torch.LongTensor)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(val_data_loader)
            epoch_val_loss += val_loss / len(val_data_loader)


    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

    torch.save(model, './vit.pt')

  0%|          | 0/213 [00:00<?, ?it/s]

Epoch : 1 - loss : 1.0350 - acc: 0.4715 - val_loss : 1.0373 - val_acc: 0.4560



  0%|          | 0/213 [00:00<?, ?it/s]

Epoch : 2 - loss : 0.9451 - acc: 0.5409 - val_loss : 1.0010 - val_acc: 0.4993



  0%|          | 0/213 [00:00<?, ?it/s]

...

Epoch : 8 - loss : 0.5408 - acc: 0.7793 - val_loss : 1.2070 - val_acc: 0.4897



  0%|          | 0/213 [00:00<?, ?it/s]

Epoch : 9 - loss : 0.4947 - acc: 0.8006 - val_loss : 1.2210 - val_acc: 0.5103



  0%|          | 0/213 [00:00<?, ?it/s]

Epoch : 10 - loss : 0.4500 - acc: 0.8207 - val_loss : 1.3656 - val_acc: 0.5247



In [22]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 17.7 MB/s eta 0:00:00


In [23]:
import torchmetrics

model.eval()
prob_all = []
lable_all = []
prob_score=[]
i=0

auroc = torchmetrics.AUROC(task='multiclass', num_classes = NUM_CLASSES)

with torch.no_grad():
  for data, label in tqdm(test_data_loader):
          data = data.to(device)
          label = label.type(torch.LongTensor)
          label = label.to(device)
          l2 = label.cpu().detach().numpy()
          lable_all.extend(l2)

          prob = model(data)

          preds_proxy = torch.sigmoid(prob).detach()

          prob = prob.cpu().detach().numpy()
          prob_score.extend(prob)
          prob_all.extend(np.argmax(prob,axis=1))



          auroc.update(preds_proxy, label)

          i=i+1
          torch.cuda.empty_cache()

#print(prob_all)
#print(label_train)
# print(i)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

# target_names = ['Hate','No']

print("Model: ", model_used )
print("Seed: ", seed )
print("Task: ", task )

print("Accuracy:{:.4f}".format(accuracy_score(lable_all, prob_all) ))
print("Recall:{:.4f}".format(recall_score(lable_all, prob_all,average='macro') ))
print("Precision:{:.4f}".format(precision_score(lable_all, prob_all,average='macro') ))
print("f1_score:{:.4f}".format(f1_score(lable_all, prob_all,average='macro') ))
print("AUC:{:.4f}".format(auroc.compute()))

print(classification_report(lable_all,prob_all,digits=4))

#fpr, tpr, thresholds = roc_curve(lable_all, prob_score, pos_label=2)
#print("Auc:{:.4f}".format(roc_auc_score(lable_all, prob_score,average='macro') ))
print(confusion_matrix(prob_all,lable_all))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Model:  vit
Seed:  510
Task:  humor
Accuracy:0.5528
Recall:0.5408
Precision:0.5564
f1_score:0.5427
AUC:0.7249
              precision    recall  f1-score   support

           0     0.7869    0.6038    0.6833       159
           1     0.3947    0.4245    0.4091       106
           2     0.4877    0.5940    0.5356       133

    accuracy                         0.5528       398
   macro avg     0.5564    0.5408    0.5427       398
weighted avg     0.5824    0.5528    0.5609       398

[[96 12 14]
 [29 45 40]
 [34 49 79]]


In [24]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# conf_mat=confusion_matrix(prob_all,lable_all)
# class_label=train_img_data.classes
# df_cm = pd.DataFrame(conf_mat, index=class_label, columns=class_label)
# df_cm
# #conf_mat
# heatmap=sns.heatmap(df_cm, annot=True, fmt="d",cmap="YlGnBu")
# heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(),rotation=0, ha='right')
# heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(),rotation=45, ha='right')